<a href="https://colab.research.google.com/github/ArcturusMajere/blsAPI/blob/master/ohywH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import oracledb;import polars as pl;import getpass

password = getpass.getpass("Gimme creds: ")
host= "host/port/service/oracle"
connection = oracledb.connect(user="FOOL", password=password,dsn=host)
def query_data(year, quarter, state, user, password, host):
    try:
        connection = oracledb.connect(user=user, password=password, dsn=dsn)
        #connection = oracledb.connect("hr/userpwd@localhost/orclpdb")
        #connection = oracledb.connect("hr/userpwd@host")
        query = f"""
        SELECT * FROM your_table
        WHERE year = :year AND quarter = :quarter AND state = :state
        """
        with connection.cursor() as cursor:
            cursor.execute(query, [year, quarter, state])
            result = cursor.fetchall()
            columns = [col[0] for col in cursor.description]

        df = pl.DataFrame(result, schema=columns)
        return df
    except Exception as e:
        print(f"HAHA give up cuz: {e}")
    finally:
        connection.close()


In [ ]:
import oracledb
import polars as pl
from sshfs import mount

def query_and_merge_data(year, quarter, state, user, password, dsn, remote_host, remote_user, remote_password, remote_path, local_mount_path):
    """
    Query database, incorporate data from a remote Parquet file, and merge dataframes by ID.

    Args:
    year (int): The year to filter the data.
    quarter (int): The quarter to filter the data.
    state (str): The state to filter the data.
    user (str): Database username.
    password (str): Database password.
    dsn (str): Data Source Name for Oracle connection.
    remote_host (str): Hostname of the remote server.
    remote_user (str): Username for SSH to the remote server.
    remote_password (str): Password for SSH to the remote server.
    remote_path (str): Path to the directory on the remote server containing the Parquet file.
    local_mount_path (str): Local path to mount the remote directory.

    Returns:
    pl.DataFrame: Data frame containing the merged data.
    """
    try:
        # Mount the remote directory
        with mount(remote_host, remote_user, password=remote_password, mountpoint=local_mount_path):
            # Read a Parquet file from the mounted directory
            remote_data = pl.read_parquet(f"{local_mount_path}/yourfile.parquet")

        # Connect to the Oracle database
        connection = oracledb.connect(user=user, password=password, dsn=dsn)

        # Define the SQL query
        query = f"""
        SELECT * FROM your_table_name
        WHERE year = :year AND quarter = :quarter AND state = :state
        """

        # Execute the query and fetch the results
        with connection.cursor() as cursor:
            cursor.execute(query, [year, quarter, state])
            result = cursor.fetchall()
            columns = [col[0] for col in cursor.description]

        # Create a Polars DataFrame from database results
        db_data = pl.DataFrame(result, schema=columns)

        # Merge DataFrames on the 'ID' column
        merged_data = db_data.join(remote_data, on="ID", how="inner")

        return merged_data
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the connection if it is open
        if 'connection' in locals():
            connection.close()

# Example usage
dsn = "host/port/service_name"
remote_host = "example.com"
remote_user = "remote_username"
remote_password = "remote_password"
remote_path = "/path/to/remote/directory"
local_mount_path = "/local/mount/point"

merged_data = query_and_merge_data(2022, 1, 'NY', 'username', 'password', dsn, remote_host, remote_user, remote_password, remote_path, local_mount_path)
print(merged_data)


In [ ]:
import oracledb
import polars as pl
from sshfs import mount

def LOOK(year, quarter, state, user, password, oraclehost, parquet_host, SSH_user, SSH_password, remote_path, local_mount_path):

    try:
        with mount(remote_host, SSH_user, password=SSH_password, mountpoint=parquet_mount_path):
            remote_data = pl.read_parquet(f"{local_mount_path}/file.parquet")

        connection = oracledb.connect(user=user, password=password, dsn=oraclehost)

        query = f"""
        SELECT * FROM your_table
        WHERE year = :year AND quarter = :quarter AND state = :state
        """
        with connection.cursor() as cursor:
            cursor.execute(query, [year, quarter, state])
            result = cursor.fetchall()
            columns = [col[0] for col in cursor.description]
        db_data = pl.DataFrame(result, schema=columns)
        merged_data = db_data.join(remote_data, on="ID", how="inner")

        return merged_data
    except Exception as e:
        print(f"HAHA fool: {e}")
    finally:
        if 'connection' in locals():
            connection.close()


LOOK(2022, 1, 'NY', 'username', 'password', dsn, remote_host, remote_user, remote_password, remote_path, local_mount_path)

